In [1]:
from src.sql_queries import fact, staging_tables, dim_tables, bridge_link
import json
import pandas as pd
from sodapy import Socrata
from IPython.core.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell
import psycopg2
InteractiveShell.ast_node_interactivity = "all"
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_colwidth', None)

In [2]:
with open('credentials.json') as f:
    data = json.load(f)
    socrates_id = data['socrates_id']
    socrates_key = data['socrates_api_key']
    app_token = data['app_token']
    
inventory_db = "6vkj-f5xf"
loans_db = "5src-czff"
codes_db = "pbt3-ytbc"
client = Socrata("data.seattle.gov", app_token, username=socrates_id,password=socrates_key, timeout=100)



In [10]:
def download_data(client_connection,database,params, limit = 50000):
    if 'where' in params:
        where_cond = params['where']
        print(where_cond)
    else:
        where_cond = None
    #first, get a count of the rows:
    target_count = client_connection.get(database, where=where_cond, select = 'count(*)')
    target_count = int(target_count[0]['count']) #since responses are always encoded as strings
    counter = 0
    all_df=[]
    while counter < target_count:
        results = client_connection.get(database, limit=limit, offset=counter, **params)
        temp_df = pd.DataFrame.from_records(results)
        all_df.append(temp_df)
        counter+=limit
    master_df = pd.concat(all_df)
    return master_df

In [21]:
where = 'checkoutdatetime > "2020-01-03T00:00:00.000" and checkoutdatetime < "2020-01-03T23:59:59.000"'
params = {'where':where}
another_df = download_data(client, loans_db, params)

checkoutdatetime > "2020-01-03T00:00:00.000" and checkoutdatetime < "2020-01-03T23:59:59.000"


download codes from API:
    every month
    df = download_data(client, codes_db,{})
    file_name = 'raw_codes_{}.csv'.format(datetime.datetime.now().strftime('%Y-%m-%d'))
    df.to_csv('./raw/{}'.format(file_name), sep='|', index=False)

upload to s3:
    check that item do not exist
    upload

upload this to redshift:
    cur.create table
    cur.commit to staging
    
delete last month's item from s3

download items from API:
    every month

conn.autocommit = True
cur = conn.cursor()


stage to dim and facts:
    cur.execute(alter table dim_ collection_backup)

In [37]:
import datetime
datetime.datetime.now().strftime('%Y-%m-%d')

'2020-05-22'